<a href="https://colab.research.google.com/github/ds4geo/ds4geo/blob/master/DS4GEO_L2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Data Science for Geoscientists - Winter Semester 2020**
# **Session 2**

In the previous session, we handled data in a very simple way using pandas. In this session we will introduce a few other helpful python object types for handling data, and expecially learn how to index/slice data (extract only certain parts of the data/object). Specifically, we will cover lists, dictionaries, and arrays from the numpy library.

We will also introduce simple array operations and aggregations, then apply these topics to a worked example from the geosciences.





# Section 1 - Lists, Dictionaries and Indexing

Lists and dictionaries are built-in python objects useful for storing and handling data.

# Lists
Python lists are ordered collections of other python objects, separated by commas. They are defined by square brackets [ ]

In [ ]:
a = [1,2,3] # List of integers
print("a:", a)

b = [1.5, 2.5, 3.5] # List of floats
print("b:", b)

In [ ]:
# Lists can contain different types
c = [1, "data", 2.5]
print("b:", b)

# Including other lists (nested)
d = [[1,2,3], [4,5,6]]
print("d:", d)

e = [a, b]
print("e:", e)

In [ ]:
# They can contain any other python objects
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
f = [pd, np, plt] # But there's no reason to actually do this
print("f:", f)

In [ ]:
# From last week, you'll recall dir() can be used to find methods on objects
a = [1, 2, 3]
# Append adds a new item to the end of a list
a.append(4)
print("a:", a)

# Extend joins to lists *in place*
a.extend(b) # notice we don't assign a result
print("a:", a)
# + operator when applied to two lists but not *in place*:
h = a + b
print("h:", h)

# sort does what it suggests, in place
a.sort()
print("a:", a)

In [ ]:
# Tuples are another type very similar to lists except they can't be modified
# i.e. you cannot append to a tuple
# They are defined by parentheses ( ) instead of [ ]
a_tuple = (1, 2, 3)
print("a_tuple:", a_tuple)

# The specific reasons for using tuples complex.
# You will see them in documentation, but usually you can just use a list

# Dictionaries
Python dictionaries are un-ordered collections of pairs known as keys and values. They function like language dictionaries where you look up a word (they key) and see its definition or translation (value).
They are defined with braces { }, separated by commas, and colons : indicate the key-value relationships.

# List and Dictionary Indexing
You can select objects/data from lists and dictionaries using square brackets [ ].
List indexing is based on numeric positions, while dictionary indexing is based on its keys.

# Section 2 - Numpy 1

Overview, creating arrays, and array operations



# Section 3 - Numpy 2

Aggregations and Indexing

# Section 4 - LA-ICPMS data reduction excercise

Motivation and overview